In [ ]:
# Important Libraries
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc

In [ ]:
# Importing customized library for FD
import FDPDash
from FDPDash import *

In [ ]:
# For setting dashboard layout
external_stylesheets = [dbc.themes.BOOTSTRAP] 

# Initializing the app
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

In [ ]:
# App layout
app.layout = html.Div([
    html.H1("Field Develpment Model", style={'textAlign': 'center'}),
    html.P("""This is an interactive dashboard to model Recovery through Waterflood and different EOR techniques, 
               and perfoem economic Analysis based on different Production Profiles by generating Petroleum Project 
               Net Cash Flows (NCF), Net Present Value (NPV), Internal Rate of Return (IRR) and Different plots for 
               Sensitivity Analysis through Tornado Charts and Spider Plots based on given CAPEX, OPEX, Tax, Royalty, 
               and Oil Prices."""),
    dbc.Row([
        dbc.Col([
            html.H5('WaterFlooding',style={'textAlign': 'center','font-weight': 'bold'}),
            dash_table.DataTable(
                id='wfdata',
                data=[
                    {'rp':'Initial Water Saturation, Swi, %','dt':0.2},
                    {'rp':'Initial Oil Saturation, Soi, %','dt':0.8},
                    {'rp':'Residual Oil Saturation to Water, Sorw, %','dt':0.25},
                    {'rp':'Oil Viscosity, Muo, cP','dt':40},
                    {'rp':'Water Viscosity, Muw, cP','dt':1},
                ],
                columns=[{'name': 'Reservoir & Fluid Parameters','id': 'rp'},
                         {'name': 'Data','id': 'dt'}],
                style_header={'textAlign': 'left'},
                style_cell={'textAlign': 'left'},
                editable=True
            )
        ],
            width="auto"),
        dcc.Store(id = 'wfdata_store',data='number'),
        dcc.Store(id = 'wfdata_frame',data='number'),
        dbc.Col(
            html.Div(dcc.Graph(id = 'relperm')),
            width="auto"
        )
    ],
        align="center",
    justify="evenly"),
    dbc.Row([
        dbc.Col(
            dcc.Graph(
            id = 'ff_rec_curves'
            ),
            width=8
        )
    ],
        align="center",
    justify="evenly"),        
    dbc.Row([
        dbc.Col([
            html.H5('Enhanced Oil Recovery',style={'textAlign': 'center','font-weight': 'bold'}),
            dash_table.DataTable(
                id='eordata',
                data=[
                    {'rp':'Initial Water Saturation, Swi, %','dt':0.2},
                    {'rp':'Initial Oil Saturation, Soi, %','dt':0.8},
                    {'rp':'Residual Oil Saturation to Water, Sorw, %','dt':0.15},
                    {'rp':'Oil Viscosity, Muo, cP','dt':40},
                    {'rp':'Water Viscosity, Muw, cP','dt':40},
                ],
                columns=[{'name': 'Reservoir & Fluid Parameters','id': 'rp'},
                         {'name': 'Data','id': 'dt'}],
                style_header={'textAlign': 'left'},
                style_cell={'textAlign': 'left'},
                editable=True
            )
        ],
            width="auto"),
        dcc.Store(id = 'eordata_store',data='number'),
        dcc.Store(id = 'eordata_frame',data='number'),
    ],
        align="center",
    justify="evenly"),
    dbc.Row([
        dbc.Col(
            dcc.Graph(
            id = 'eor_curves'
            ),
            width=8
        )
    ],
        align="center",
    justify="evenly"),
    html.Div(id='info'),
    dbc.Row([
        dbc.Col([
            html.H5('Field Data',style={'textAlign': 'center','font-weight': 'bold'}),
            dash_table.DataTable(
                id='field_data',
                data=[
                    {'fp':'Field Area, Km2','dt':20},
                    {'fp':'Oil Initially in Place, MMm3','dt':25},
                    {'fp':'Test Liq. Rate, m3/d','dt':25},
                    {'fp':'Years on Production','dt':25},
                ],
                columns=[{'name': 'Field Parameters','id': 'fp'},
                         {'name': 'Data','id': 'dt'}],
                style_header={'textAlign': 'left'},
                style_cell={'textAlign': 'left'},
                editable=True
            )
        ],
            width="auto"),
        dcc.Store(id = 'field_data_store',data='number'),
        dcc.Store(id = 'coef_wf_store',data='number'),
        dcc.Store(id = 'intcpt_wf_store',data='number'),
        dcc.Store(id = 'coef_eor_store',data='number'),
        dcc.Store(id = 'intcpt_eor_store',data='number'),
        dcc.Store(id = 'wf_prod_pfile_frame',data='number'),
        dcc.Store(id = 'eor_prod_pfile_frame',data='number'),
        dbc.Col([
            html.H5('Economic Variables',style={'textAlign': 'center','font-weight': 'bold'}),
            dash_table.DataTable(
                id='exp_data',
                data=[
                    {'ev':'CAPEX, Mn$','dt':100},
                    {'ev':'OPEX, $/bbl','dt':40},
                    {'ev':'Well Cost, Mn$/well','dt':0.25},
                    {'ev':'Oil Price, $/bbl','dt':60},
                    {'ev':'Discount Rate, % p.a','dt':0.14},
                    {'ev':'Royalty, % Gross Rev.','dt':0.15},
                    {'ev':'Tax, % p.a.','dt':0.2}
                ],
                columns=[{'name': 'Economic Variables','id': 'ev'},
                         {'name': 'Data','id': 'dt'}],
                style_header={'textAlign': 'left'},
                style_cell={'textAlign': 'left'},
                editable=True
            )
        ],
            width=2),
        dcc.Store(id = 'exp_data_store',data='number')
        ],
        align="center",
    justify="evenly"),
    dbc.Row([
        dbc.Col([
            html.Div(dcc.Graph(id = 'wf_stack')),
            html.P("Spacing:"),
            dcc.Slider(
                id='wf_spacing_stack', 
                min=200, 
                max=700, 
                value=300, 
                step=100)
        ],
            width=4
        ),
        dbc.Col([
            html.Div(dcc.Graph(id = 'eor_stack')),
            html.P("Spacing:"),
            dcc.Slider(
                id='eor_spacing_stack', 
                min=200, 
                max=700, 
                value=300, 
                step=100)
        ],
            width=4
        )
    ],
        align="center",
    justify="evenly"),
    dbc.Row([
        dbc.Col(
            dcc.Graph(
            id = 'wf_eco_curves'
            ),
            width=10
        )
    ],
        align="center",
    justify="evenly"),
    dbc.Row([
        dbc.Col(
            dcc.Graph(
            id = 'eor_eco_curves'
            ),
            width=10
        )
    ],
        align="center",
    justify="evenly"),
    dbc.Row([
        dbc.Col([
            html.H5('Master Table - Waterflooding',style={'textAlign': 'center','font-weight': 'bold'}),
            dash_table.DataTable(
                id='master_table_wf',
                style_header={'textAlign': 'left'},
                style_cell={'textAlign': 'left'},
                editable=False
            )
        ],
            width=4),
        dbc.Col([
            html.Div(dcc.Graph(id = 'cashflow_bar_wf')),
            html.P("Waterflooding - Spacing:"),
            dcc.Slider(
                id='wf_spacing_eco', 
                min=200, 
                max=700, 
                value=300, 
                step=100)
        ],
            width=4
        )
    ],
        align="center",
    justify="evenly"),
    html.Div(id='best_spacing_wfinfo'),
    dbc.Row([
        dbc.Col(
            dcc.Graph(
            id = 'sensitivity_chart_wf'
            ),
            width=8
        )
    ],
        align="center",
    justify="evenly"),
    dbc.Row([
        dbc.Col([
            html.H5('Master Table - Enhanced Oil Recovery',style={'textAlign': 'center','font-weight': 'bold'}),
            dash_table.DataTable(
                id='master_table_eor',
                style_header={'textAlign': 'left'},
                style_cell={'textAlign': 'left'},
                editable=False
            )
        ],
            width="auto"),
        dbc.Col([
            html.Div(dcc.Graph(id = 'cashflow_bar_eor')),
            html.P("EOR - Spacing:"),
            dcc.Slider(
                id='eor_spacing_eco', 
                min=200, 
                max=700, 
                value=300, 
                step=100)
        ],
            width=4
        )
    ],
        align="center",
    justify="evenly"),
    html.Div(id='best_spacing_eorinfo'),
    dbc.Row([
        dbc.Col(
            dcc.Graph(
            id = 'sensitivity_chart_eor'
            ),
            width=8
        )
    ],
        align="center",
    justify="evenly"),
    dbc.Row([
        dbc.Col(
            dcc.Graph(
            id = 'cashflow_clustered'
            ),
            width="auto"
        ),
        dbc.Col([
            html.P("WF - Spacing:"),
            dcc.Slider(
                id='wf_spacing_bar', 
                min=200, 
                max=700, 
                value=300, 
                step=100),
            html.P("EOR - Spacing:"),
            dcc.Slider(
                id='eor_spacing_bar', 
                min=200, 
                max=700, 
                value=300, 
                step=100)
        ],
            width=3
        )
    ],
        align="center",
    justify="evenly")
])

In [ ]:
# App Callbacks
@app.callback([Output('wfdata_store', 'data'),
                Output('wfdata','data')],
                Input('wfdata', 'data'))
def save_wf_data(data):
    if (float(data[0]['dt'])>1 or float(data[1]['dt'])>1):
        raise PreventUpdate
    if(type(data[1]['dt'])==float):
        data[0]['dt']=float(data[0]['dt'])
        data[1]['dt']=float(1-data[0]['dt'])
    else:
        data[1]['dt']=float(data[1]['dt'])
        data[0]['dt']=float(1-data[1]['dt'])
    for i in range(len(data)):
        data[i]['dt']=round(float(data[i]['dt']),3)
    return data,data

@app.callback(Output('wfdata_frame', 'data'),
               Input('wfdata_store', 'data'))
def save_wf_dataframe(data):
    wf_data = Rec_Analysis(float(data[0]['dt']),float(data[2]['dt']),float(data[3]['dt']),float(data[4]['dt'])).to_dict('records')
    return wf_data

@app.callback([Output('relperm', 'figure'),
                Output('ff_rec_curves', 'figure')],
              Input('wfdata_frame', 'data'))
def wf_curve(wf_data):
    if wf_data is None:
        raise PreventUpdate
    wf_data = pd.DataFrame(wf_data)
    return Rel_Perm_Curve(wf_data),Curves(wf_data,"Waterflooding")

@app.callback([Output('eordata_store', 'data'),
                Output('eordata','data')],
                Input('eordata', 'data'))
def save_eor_data(data):
    if (float(data[0]['dt'])>1 or float(data[1]['dt'])>1):
        raise PreventUpdate
    if(type(data[1]['dt'])==float):
        data[0]['dt']=float(data[0]['dt'])
        data[1]['dt']=float(1-data[0]['dt'])
    else:
        data[1]['dt']=float(data[1]['dt'])
        data[0]['dt']=float(1-data[1]['dt'])
    for i in range(len(data)):
        data[i]['dt']=round(float(data[i]['dt']),3)
    return data,data

@app.callback(Output('eordata_frame', 'data'),
               Input('eordata_store', 'data'))
def save_eor_dataframe(data):
    eor_data = Rec_Analysis(float(data[0]['dt']),float(data[2]['dt']),float(data[3]['dt']),float(data[4]['dt'])).to_dict('records')
    return eor_data

@app.callback([Output('eor_curves', 'figure'),
                Output('info', 'children')],
               [Input('wfdata_frame', 'data'),
                Input('eordata_frame', 'data')])
def eor_curve(wf_data,eor_data):
    if wf_data is None:
        raise PreventUpdate
    if eor_data is None:
        raise PreventUpdate
    wf_data = pd.DataFrame(wf_data)
    eor_data = pd.DataFrame(eor_data)
    fig, text = EOR_Curves(wf_data,eor_data)
    return fig, html.P(text)

@app.callback([Output('field_data_store', 'data'),
                Output('field_data','data')],
                Input('field_data', 'data'))
def save_field_data(data):
    for i in range(len(data)):
        data[i]['dt']=round(float(data[i]['dt']),3)
    field_pmtrs = Prod_dictionary(data[0]['dt'],data[1]['dt'],data[2]['dt'],data[3]['dt'])
    return field_pmtrs,data

@app.callback([Output('coef_wf_store', 'data'),
               Output('intcpt_wf_store', 'data'),
               Output('coef_eor_store', 'data'),
               Output('intcpt_eor_store', 'data')],
               [Input('wfdata_frame', 'data'),
               Input('eordata_frame', 'data')])
def coef_intcpt(wf_data,eor_data):
    wf_data = pd.DataFrame(wf_data)
    eor_data = pd.DataFrame(eor_data)
    coef_wf,intercept_wf = Poly_eq(6,wf_data.iloc[:,0:1].values,wf_data.iloc[:,-1:].values)
    coef_eor,intercept_eor = Poly_eq(6,eor_data.iloc[:,0:1].values,eor_data.iloc[:,-1:].values)
    return list(coef_wf),list(intercept_wf),list(coef_eor),list(intercept_eor)

@app.callback([Output('exp_data_store', 'data'),
                Output('exp_data','data')],
                Input('exp_data', 'data'))
def save_exp_data(data):
    for i in range(len(data)):
        data[i]['dt']=round(float(data[i]['dt']),3)
    exp_pmtrs = Exp_dictionary(data[0]['dt'],data[1]['dt'],data[2]['dt'],data[3]['dt'],data[4]['dt'],data[5]['dt'],data[6]['dt'])
    return exp_pmtrs, data

@app.callback([Output('wf_prod_pfile_frame', 'data'),
               Output('eor_prod_pfile_frame', 'data'),
               Output('wf_eco_curves', 'figure'),
               Output('eor_eco_curves', 'figure')],
               [Input('field_data_store', 'data'),
                Input('exp_data_store', 'data'),
                Input('coef_wf_store', 'data'),
                Input('intcpt_wf_store', 'data'),
                Input('coef_eor_store', 'data'),
                Input('intcpt_eor_store', 'data'),
                Input("wf_spacing_stack", "value"),
                Input("eor_spacing_stack", "value")])
def save_dataframe(field_pmtrs,exp_pmtrs,coef_wf,intercept_wf,coef_eor,intercept_eor,wf_value_stack,eor_value_stack):#,wf_value_eco,eor_value_eco,wf_value_bar,eor_value_bar):
#     coef_wf = np.array(coef_wf) 
#     intercept_wf = np.array(intercept_wf)
#     coef_eor = np.array(coef_eor)
#     intercept_eor = np.array(intercept_eor)
    wf_profile = Production_profile(field_pmtrs,coef_wf,intercept_wf,wf_value_stack).to_dict('records')
    eor_profile = Production_profile(field_pmtrs,coef_eor,intercept_eor,eor_value_stack).to_dict('records')
    return wf_profile, eor_profile, Eco_curves(field_pmtrs,coef_wf,intercept_wf,"Before EOR"),Eco_curves(field_pmtrs,coef_eor,intercept_eor,"After EOR")

#, master_table_wf.to_dict('records'), html.P(text1),Cashflow_bar(field_pmtrs,exp_pmtrs,coef_wf,intercept_wf,"Before EOR",wf_value_eco),Sensitivity_chart(field_pmtrs,exp_pmtrs,coef_wf,intercept_wf,"Before EOR",wf_value_eco),master_table_eor.to_dict('records'), html.P(text2),Cashflow_bar(field_pmtrs,exp_pmtrs,coef_eor,intercept_eor,"After EOR",eor_value_eco),Sensitivity_chart(field_pmtrs,exp_pmtrs,coef_eor,intercept_eor,"After EOR",eor_value_eco),Cashflow_bar_clustered(field_pmtrs,exp_pmtrs,coef_wf,intercept_wf,coef_eor,intercept_eor,wf_value_bar,eor_value_bar)

@app.callback([Output('master_table_wf', 'data'),
               Output('best_spacing_wfinfo', 'children'),
               Output('cashflow_bar_wf','figure'),
               Output('sensitivity_chart_wf','figure'),
               Output('master_table_eor', 'data'),
               Output('best_spacing_eorinfo', 'children'),
               Output('cashflow_bar_eor','figure'),
               Output('sensitivity_chart_eor','figure'),
               Output('cashflow_clustered', 'figure')],
               [Input('field_data_store', 'data'),
                Input('exp_data_store', 'data'),
                Input('coef_wf_store', 'data'),
                Input('intcpt_wf_store', 'data'),
                Input('coef_eor_store', 'data'),
                Input('intcpt_eor_store', 'data'),
                Input("wf_spacing_eco", "value"),
                Input("eor_spacing_eco", "value"),
                Input("wf_spacing_bar", "value"),
                Input("eor_spacing_bar", "value")])
def save_dataframe(field_pmtrs,exp_pmtrs,coef_wf,intercept_wf,coef_eor,intercept_eor,wf_value_eco,eor_value_eco,wf_value_bar,eor_value_bar):
    master_table_wf, text1 = Eco_analysis(field_pmtrs,exp_pmtrs,coef_wf,intercept_wf)
    master_table_wf = master_table_wf.to_dict('records')
    master_table_eor, text2 = Eco_analysis(field_pmtrs,exp_pmtrs,coef_eor,intercept_eor)
    master_table_eor = master_table_eor.to_dict('records')
    for i in range(len(master_table_wf)):
        for j in range(2,8):
            master_table_wf[i][str(j)+'00 m']=round(float(master_table_wf[i][str(j)+'00 m']),3)
            master_table_eor[i][str(j)+'00 m']=round(float(master_table_eor[i][str(j)+'00 m']),3)
    return master_table_wf, html.P(text1),Cashflow_bar(field_pmtrs,exp_pmtrs,coef_wf,intercept_wf,"Before EOR",wf_value_eco),Sensitivity_chart(field_pmtrs,exp_pmtrs,coef_wf,intercept_wf,"Before EOR",wf_value_eco), master_table_eor, html.P(text2), Cashflow_bar(field_pmtrs,exp_pmtrs,coef_eor,intercept_eor,"After EOR",eor_value_eco),Sensitivity_chart(field_pmtrs,exp_pmtrs,coef_eor,intercept_eor,"After EOR",eor_value_eco),Cashflow_bar_clustered(field_pmtrs,exp_pmtrs,coef_wf,intercept_wf,coef_eor,intercept_eor,wf_value_bar,eor_value_bar)

@app.callback([Output('wf_stack', 'figure'),
               Output('eor_stack', 'figure')],
               [Input('wf_prod_pfile_frame', 'data'),
                Input('eor_prod_pfile_frame', 'data')])
def avg_rate_stack_curve(wf_profile,eor_profile):
    if wf_profile is None:
        raise PreventUpdate
    if eor_profile is None:
        raise PreventUpdate
    wf_profile = pd.DataFrame(wf_profile)
    eor_profile = pd.DataFrame(eor_profile)
    return Avg_rate_stacked_bar(wf_profile,"Before EOR"), Avg_rate_stacked_bar(eor_profile,"After EOR")

In [ ]:
# Running the app
app.run_server(debug=True)